### 🔎 Filtros y Consultas Avanzadas

Ya sabes cómo explorar un `DataFrame` y seleccionar subconjuntos por su posición o etiqueta. Ahora, daremos un paso más allá para aprender a hacer una de las tareas más comunes y poderosas en el análisis de datos: **filtrar filas basadas en condiciones lógicas**.

Aquí es donde dejas de solo *ver* los datos y empiezas a *interrogarlos*.

* **3.1. Máscaras Booleanas: El Corazón del Filtrado**

El concepto fundamental detrás del filtrado en Pandas es la **máscara booleana**. Es más simple de lo que suena: es una Serie de Pandas que contiene únicamente valores `True` o `False`.

Cuando aplicas una condición a una Serie (por ejemplo, `df['columna'] > 10`), Pandas te devuelve una de estas máscaras.

In [1]:
import pandas as pd

# Volvemos a cargar nuestro dataset de pingüinos
url = 'https://raw.githubusercontent.com/allisonhorst/palmerpenguins/main/inst/extdata/penguins.csv'
df = pd.read_csv(url)

# Creemos nuestra primera máscara booleana
# Condición: ¿Qué pingüinos son de la especie "Adelie"?
mascara_adelie = df['species'] == 'Adelie'

print("Esto es una máscara booleana (una Serie de True/False):")
display(mascara_adelie.head())

print(f"\nLa máscara tiene {mascara_adelie.sum()} valores 'True'.")

Esto es una máscara booleana (una Serie de True/False):


0    True
1    True
2    True
3    True
4    True
Name: species, dtype: bool


La máscara tiene 152 valores 'True'.


Para aplicar el filtro, simplemente pasas esta máscara dentro de los corchetes `[]` de tu DataFrame. Pandas te devolverá solo las filas donde la máscara tenga el valor `True`.

In [2]:
# Aplicando la máscara al DataFrame
df_adelie = df[mascara_adelie]

print("DataFrame filtrado solo con pingüinos 'Adelie':")
display(df_adelie.head())

print(f"Dimensiones del DataFrame original: {df.shape}")
print(f"Dimensiones del DataFrame filtrado: {df_adelie.shape}")

DataFrame filtrado solo con pingüinos 'Adelie':


,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,year
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,male,2007
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,female,2007
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,female,2007
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN,2007
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,female,2007


Dimensiones del DataFrame original: (344, 8)
Dimensiones del DataFrame filtrado: (152, 8)


* **3.2. Componiendo Filtros: `&` (y), `|` (o), `~` (no)**

Rara vez querrás filtrar por una sola condición. Para combinar múltiples condiciones, usamos los siguientes operadores:

* `&` para **Y** (ambas condiciones deben ser verdaderas).
* `|` para **O** (al menos una de las condiciones debe ser verdadera).
* `~` para **NO** (niega una condición).

**¡REGLA DE ORO!** Cada condición individual **DEBE** estar envuelta en paréntesis `()` debido a las reglas de precedencia de operadores en Python.

In [3]:
# Condición 1: Pingüinos de la especie 'Gentoo'
# Condición 2: Cuya masa corporal sea superior a 5000g

# Usando el operador & (Y)
df_gentoo_grandes = df[(df['species'] == 'Gentoo') & (df['body_mass_g'] > 5000)]
display(df_gentoo_grandes.head())

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,year
153,Gentoo,Biscoe,50.0,16.3,230.0,5700.0,male,2007
155,Gentoo,Biscoe,50.0,15.2,218.0,5700.0,male,2007
156,Gentoo,Biscoe,47.6,14.5,215.0,5400.0,male,2007
159,Gentoo,Biscoe,46.7,15.3,219.0,5200.0,male,2007
161,Gentoo,Biscoe,46.8,15.4,215.0,5150.0,male,2007


In [4]:
# Condición 1: Pingüinos de la isla 'Dream'
# Condición 2: O pingüinos de la isla 'Biscoe'
# Esto se puede hacer más fácil con .isin(), ¡que veremos luego!
df_dream_o_biscoe = df[(df['island'] == 'Dream') | (df['island'] == 'Biscoe')]
display(df_dream_o_biscoe.tail())

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,year
339,Chinstrap,Dream,55.8,19.8,207.0,4000.0,male,2009
340,Chinstrap,Dream,43.5,18.1,202.0,3400.0,female,2009
341,Chinstrap,Dream,49.6,18.2,193.0,3775.0,male,2009
342,Chinstrap,Dream,50.8,19.0,210.0,4100.0,male,2009
343,Chinstrap,Dream,50.2,18.7,198.0,3775.0,female,2009


In [5]:
# Usando el operador ~ (NO)
# Todos los pingüinos que NO son machos
df_no_machos = df[~(df['sex'] == 'male')]
display(df_no_machos.head())

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,year
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,female,2007
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,female,2007
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN,2007
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,female,2007
6,Adelie,Torgersen,38.9,17.8,181.0,3625.0,female,2007


* **3.3. Consultas Más Legibles con `.query()`**

Escribir muchas condiciones con `&` y `|` puede volverse difícil de leer. El método `.query()` es una alternativa elegante que te permite escribir tus filtros en un formato de texto (string), que a menudo es más intuitivo.

In [6]:
# La misma consulta de los 'Gentoo' grandes, pero con .query()
df_gentoo_grandes_query = df.query("species == 'Gentoo' and body_mass_g > 5000")
display(df_gentoo_grandes_query.head())

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,year
153,Gentoo,Biscoe,50.0,16.3,230.0,5700.0,male,2007
155,Gentoo,Biscoe,50.0,15.2,218.0,5700.0,male,2007
156,Gentoo,Biscoe,47.6,14.5,215.0,5400.0,male,2007
159,Gentoo,Biscoe,46.7,15.3,219.0,5200.0,male,2007
161,Gentoo,Biscoe,46.8,15.4,215.0,5150.0,male,2007


In [7]:
# La consulta de las islas con 'or'
df_dream_o_biscoe_query = df.query("island == 'Dream' or island == 'Biscoe'")
display(df_dream_o_biscoe_query.tail())

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,year
339,Chinstrap,Dream,55.8,19.8,207.0,4000.0,male,2009
340,Chinstrap,Dream,43.5,18.1,202.0,3400.0,female,2009
341,Chinstrap,Dream,49.6,18.2,193.0,3775.0,male,2009
342,Chinstrap,Dream,50.8,19.0,210.0,4100.0,male,2009
343,Chinstrap,Dream,50.2,18.7,198.0,3775.0,female,2009


**Ventaja de `.query()`:** Puedes hacer referencia a variables externas usando el símbolo `@`.

In [8]:
masa_limite = 3500
especie_objetivo = 'Chinstrap'

# Filtramos usando variables externas
df_filtrado_variables = df.query("body_mass_g < @masa_limite and species == @especie_objetivo")
display(df_filtrado_variables)

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,year
282,Chinstrap,Dream,46.1,18.2,178.0,3250.0,female,2007
292,Chinstrap,Dream,50.3,20.0,197.0,3300.0,male,2007
294,Chinstrap,Dream,46.4,18.6,190.0,3450.0,female,2007
297,Chinstrap,Dream,48.5,17.5,191.0,3400.0,male,2007
298,Chinstrap,Dream,43.2,16.6,187.0,2900.0,female,2007
300,Chinstrap,Dream,46.7,17.9,195.0,3300.0,female,2007
302,Chinstrap,Dream,50.5,18.4,200.0,3400.0,female,2008
306,Chinstrap,Dream,40.9,16.6,187.0,3200.0,female,2008
308,Chinstrap,Dream,42.5,16.7,187.0,3350.0,female,2008
314,Chinstrap,Dream,46.9,16.6,192.0,2700.0,female,2008


* **🧠 Ejercicios Propuestos**

De nuevo con el dataset del Titanic. ¡A interrogar los datos!
`url_titanic = 'https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv'`

1. **Carga los datos del Titanic.**

2. **Filtro Simple:** Crea un DataFrame que contenga únicamente a los pasajeros que **sobrevivieron** (`Survived == 1`). ¿Cuántos fueron?

3. **Filtro Compuesto con `&`:** Filtra los datos para encontrar a todas las **mujeres** (`Sex == 'female'`) de **primera clase** (`Pclass == 1`). ¿Cuántas de ellas sobrevivieron? (Necesitarás un segundo paso para esto).

4. **Filtro Compuesto con `|`:** Crea un DataFrame que contenga a todos los pasajeros que eran **menores de 18 años** O **mayores de 60 años**.

5. **Desafío con `.query()`:** Repite el ejercicio 3 (mujeres de primera clase) pero esta vez usando el método `.query()`.